In [1]:
import pandas as pd
import numpy as np
import itertools
import datetime
import matplotlib.pyplot as plt
from datetime import *
from datetime import datetime, timedelta, date
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
project_id = "perceptive-ivy-290216"

# Standard plotly imports
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [3]:
query2=f"""
SELECT
A.*,
B.RoundNumber
FROM `perceptive-ivy-290216.f1_api.results_race` A
INNER JOIN
`perceptive-ivy-290216.f1_api.schedules_update` B
ON A.GP=B.EventName AND A.Year=B.Year
WHERE
A.Year=2024
AND B.Year=2024
-- AND
--  A.GP='Qatar Grand Prix'
"""

track2=pd.read_gbq(query2,project_id,dialect='standard')

/var/folders/x_/b65sxrpx6737wtqnt0ctv71w0000gn/T/ipykernel_12999/1015015979.py:16: FutureWarning: read_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.read_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.read_gbq
  track2=pd.read_gbq(query2,project_id,dialect='standard')


In [4]:
query3=f"""
SELECT
A.*,
B.RoundNumber
FROM `perceptive-ivy-290216.f1_api.results_sprint` A
INNER JOIN
`perceptive-ivy-290216.f1_api.schedules_update` B
ON A.GP=B.EventName AND A.Year=B.Year
WHERE
A.Year=2024
-- AND
-- A.GP='Qatar Grand Prix'
"""
track3=pd.read_gbq(query3,project_id,dialect='standard')

/var/folders/x_/b65sxrpx6737wtqnt0ctv71w0000gn/T/ipykernel_12999/3282991377.py:14: FutureWarning: read_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.read_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.read_gbq
  track3=pd.read_gbq(query3,project_id,dialect='standard')


In [5]:
#Points

In [6]:
points_table=track2[["RoundNumber","GP","Abbreviation","Points"]]
points_table["Points"]=points_table["Points"].astype(int)
points_table=points_table.sort_values(by=["RoundNumber","Abbreviation"])
points_table.head()

/var/folders/x_/b65sxrpx6737wtqnt0ctv71w0000gn/T/ipykernel_12999/3640686471.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  points_table["Points"]=points_table["Points"].astype(int)


,RoundNumber,GP,Abbreviation,Points
140,1,Bahrain Grand Prix,ALB,0
151,1,Bahrain Grand Prix,ALO,2
169,1,Bahrain Grand Prix,BOT,0
181,1,Bahrain Grand Prix,GAS,0
199,1,Bahrain Grand Prix,HAM,6


In [7]:
points_table_sprint=track3[["RoundNumber","GP","Abbreviation","Points"]]
points_table_sprint["Points"]=points_table_sprint["Points"].astype(int)
points_table_sprint=points_table_sprint.sort_values(by=["RoundNumber","Abbreviation"])

/var/folders/x_/b65sxrpx6737wtqnt0ctv71w0000gn/T/ipykernel_12999/482007640.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  points_table_sprint["Points"]=points_table_sprint["Points"].astype(int)


In [8]:
points_table=pd.concat([points_table,points_table_sprint])
points_table.head()

,RoundNumber,GP,Abbreviation,Points
140,1,Bahrain Grand Prix,ALB,0
151,1,Bahrain Grand Prix,ALO,2
169,1,Bahrain Grand Prix,BOT,0
181,1,Bahrain Grand Prix,GAS,0
199,1,Bahrain Grand Prix,HAM,6


In [9]:
#Get all drivers GP counts to fill in missing data and remove drivers who drove sparingly. The Other Option is to add the GP Data for sparing drivers to keep the dataset uniform
points_table.groupby(by=["Abbreviation"])["GP"].count()

Abbreviation
ALB    20
ALO    20
BEA     2
BOT    20
COL     2
GAS    20
HAM    20
HUL    20
LEC    20
MAG    19
NOR    20
OCO    20
PER    20
PIA    20
RIC    20
RUS    20
SAI    19
SAR    17
STR    20
TSU    20
VER    20
ZHO    20
Name: GP, dtype: int64

In [10]:
#Find out which GPs were missed by drivers and then use the next block to fill in the details missing
# points_table[points_table["RoundNumber"].isin([16])].sort_values(by=["RoundNumber","DriverId"])
points_table[points_table["Abbreviation"]=="SAI"]

,RoundNumber,GP,Abbreviation,Points
303,1,Bahrain Grand Prix,SAI,15
298,3,Australian Grand Prix,SAI,25
300,4,Japanese Grand Prix,SAI,15
301,5,Chinese Grand Prix,SAI,10
295,6,Miami Grand Prix,SAI,10
296,7,Emilia Romagna Grand Prix,SAI,10
302,8,Monaco Grand Prix,SAI,15
305,9,Canadian Grand Prix,SAI,0
297,10,Spanish Grand Prix,SAI,8
299,11,Austrian Grand Prix,SAI,15


In [11]:
points_table=points_table[(points_table["Abbreviation"]!="HUL")&(points_table["Abbreviation"]!="DEV")]

In [12]:
#Adding Albon's Italian GP Data which is missing. This messes up the line chart, so to avoid that and uniformity in data adding this row here.
points_table.loc[-1] = [2,"Saudi Arabian Grand Prix", "SAI", 0] # adding a row

In [13]:
#Combine Race and Sprint in 1 row
points_table=pd.DataFrame(points_table.groupby(by=["RoundNumber","GP","Abbreviation"])["Points"].sum()).reset_index()
points_table=points_table.sort_values(by=["Abbreviation","RoundNumber"])
points_table["cumsum"]=points_table.groupby(by=["Abbreviation"])["Points"].cumsum()

#Create a new table that has totals by driver id which will be used to sort the final chart
totals=pd.DataFrame(points_table.groupby(by=["Abbreviation"])["Points"].sum())
totals.columns=["sum"]

#Merge both tables
points_table=points_table.merge(totals, left_on='Abbreviation', right_on='Abbreviation')

points_table=points_table.sort_values(by=["sum","Abbreviation","RoundNumber"], ascending=[False, True, True])
points_table.head()

,RoundNumber,GP,Abbreviation,Points,cumsum,sum
289,1,Bahrain Grand Prix,VER,26,26,313
290,2,Saudi Arabian Grand Prix,VER,25,51,313
291,3,Australian Grand Prix,VER,0,51,313
292,4,Japanese Grand Prix,VER,26,77,313
293,5,Chinese Grand Prix,VER,33,110,313


In [14]:
fig=px.line(
    points_table,
    y="cumsum",
    x='GP',
    # text='cumsum',
    # text='LapTimeDelta2',
    color='Abbreviation',
    title="<b>Driver's Championship Points</b>",
    height=1200, width=1600    ,
    template="plotly_dark",

           color_discrete_map={
                 "ALB": "#64C4FF",
                 "ALO": "#229971",
                 "BOT": "#52e252",
                 "GAS": "#0093cc",
                 "HAM": "#27F4D2",
                 "HUL": "#B6BABD",
                 "LEC": "#E80020",
                 "MAG": "#B6BABD",
                 "NOR": "#FF8000",
                 "OCO": "#0093cc",
                 "PER": "#3671C6",
                 "PIA": "#FF8000",
                 "RIC": "#6692FF",
                 "RUS": "#27F4D2",
                 "SAI": "#E80020",
                 "SAR": "#37BEDD",
                 "STR": "#229971",
                 "TSU": "#6692FF",
                 "VER": "#3671C6",
                 "ZHO": "#52e252"
          }
)

fig.update_layout(
    title_x=0.5,
    hoverlabel=dict(
        bgcolor="white",
        font_size=16,
        font_family="PT Sans Narrow"
    ),
    xaxis_title="<b>GP</b>",
    yaxis_title="<b>POINTS</b>",
    yaxis = dict(tickfont = dict(size=15)),
    xaxis = dict(tickfont = dict(size=15)),
    font=dict(
        family="PT Sans Narrow",
        size=14,
        color="White"
    ),
    title_font_family="PT Sans Narrow"
#  "Arial", "Balto", "Courier New", "Droid Sans", "Droid Serif", "Droid Sans Mono", "Gravitas One", "Old Standard TT", "Open Sans", "Overpass", "PT Sans Narrow", "Raleway", "Times New Roman"
)

fig.show()

In [15]:
fig=px.scatter(
    points_table,
    y="Abbreviation",
    x='cumsum',
    color='Abbreviation',
    # text="cumsum",
    title="<b>Driver's Championship Points</b>",
    height=1000, width=1200,
    animation_frame="RoundNumber",
    animation_group="Abbreviation",
    range_x=[0,500],
    # range_y=[0,500],
    size='cumsum',
    # text='cumsum',

           color_discrete_map={
                 "ALB": "#64C4FF",
                 "ALO": "#229971",
                 "BOT": "#52e252",
                 "GAS": "#0093cc",
                 "HAM": "#27F4D2",
                 "HUL": "#B6BABD",
                 "LEC": "#E80020",
                 "MAG": "#B6BABD",
                 "NOR": "#FF8000",
                 "OCO": "#0093cc",
                 "PER": "#3671C6",
                 "PIA": "#FF8000",
                 "RIC": "#6692FF",
                 "RUS": "#27F4D2",
                 "SAI": "#E80020",
                 "SAR": "#37BEDD",
                 "STR": "#229971",
                 "TSU": "#6692FF",
                 "VER": "#3671C6",
                 "ZHO": "#52e252"
          }
)
fig.update_layout(
    title_x=0.5,
    hoverlabel=dict(
        bgcolor="white",
        font_size=16,
        font_family="PT Sans Narrow"
    ),
    xaxis_title="<b>POINTS</b>",
    yaxis_title="<b>DRIVER</b>",
    yaxis = dict(tickfont = dict(size=15)),
    xaxis = dict(tickfont = dict(size=15)),
    font=dict(
        family="PT Sans Narrow",
        size=14,
        color="Black"
    ),
    title_font_family="PT Sans Narrow"
#  "Arial", "Balto", "Courier New", "Droid Sans", "Droid Serif", "Droid Sans Mono", "Gravitas One", "Old Standard TT", "Open Sans", "Overpass", "PT Sans Narrow", "Raleway", "Times New Roman"
)

fig.show()

In [16]:
points_table['cumsum']=points_table['cumsum'].astype(int)

In [17]:
fig=px.bar(
    points_table,
    y="cumsum",
    x='Abbreviation',
    color='Abbreviation',
    text='cumsum',
    title="<b>Driver's Championship Points</b>",
    height=1000, width=1200,
    animation_frame="RoundNumber",
    animation_group="Abbreviation",
    # range_x=[0,500],
    range_y=[0,500],
    template="plotly_dark",
    # size='cumsum',
    # text='cumsum',

           color_discrete_map={
                 "ALB": "#64C4FF",
                 "ALO": "#229971",
                 "BOT": "#52e252",
                 "GAS": "#0093cc",
                 "HAM": "#27F4D2",
                 "HUL": "#B6BABD",
                 "LEC": "#E80020",
                 "MAG": "#B6BABD",
                 "NOR": "#FF8000",
                 "OCO": "#0093cc",
                 "PER": "#3671C6",
                 "PIA": "#FF8000",
                 "RIC": "#6692FF",
                 "RUS": "#27F4D2",
                 "SAI": "#E80020",
                 "SAR": "#37BEDD",
                 "STR": "#229971",
                 "TSU": "#6692FF",
                 "VER": "#3671C6",
                 "ZHO": "#52e252"
          }
)
fig.update_traces(textposition='outside')

fig.update_layout(
    title_x=0.5,
    hoverlabel=dict(
        bgcolor="white",
        font_size=16,
        font_family="PT Sans Narrow"
    ),
    xaxis_title="<b>DRIVER</b>",
    yaxis_title="<b>POINTS</b>",
    yaxis = dict(tickfont = dict(size=15)),
    xaxis = dict(tickfont = dict(size=15)),
    font=dict(
        family="PT Sans Narrow",
        size=14,
        color="White"
    ),
    title_font_family="PT Sans Narrow"
#  "Arial", "Balto", "Courier New", "Droid Sans", "Droid Serif", "Droid Sans Mono", "Gravitas One", "Old Standard TT", "Open Sans", "Overpass", "PT Sans Narrow", "Raleway", "Times New Roman"
)

fig.show()